# Queries in PyTables

In [6]:
import os
import numpy as np
import pandas as pd
import tables

In [4]:
%ls -lh structuring compression

compression:
total 126592
-rw-r--r--  1 faltet  staff   5.0M May 16 15:03 blosc-blosclz-5-shuffle.h5
-rw-r--r--  1 faltet  staff   5.5M May 16 15:03 blosc-lz4-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.8M May 16 15:03 blosc-lz4hc-5-shuffle.h5
-rw-r--r--  1 faltet  staff    17M May 16 15:03 blosc-snappy-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.3M May 16 15:03 blosc-zlib-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.3M May 16 15:03 blosc-zstd-5-shuffle.h5
-rw-r--r--  1 faltet  staff    17M May 16 15:03 no-compressed.h5
-rw-r--r--  1 faltet  staff   4.2M May 16 15:03 zlib-5-shuffle.h5

structuring:
total 425904
-rw-r--r--  1 faltet  staff   7.2M May 16 15:09 blosc-blosclz-5-shuffle.h5
-rw-r--r--  1 faltet  staff   7.8M May 16 15:09 blosc-lz4-5-shuffle.h5
-rw-r--r--  1 faltet  staff   6.6M May 16 15:09 blosc-lz4hc-5-shuffle.h5
-rw-r--r--  1 faltet  staff    13M May 16 15:09 blosc-snappy-5-shuffle.h5
-rw-r--r--  1 faltet  staff   6.0M May 16 15:09 blosc-zlib-5-shuffle.h5
-rw-r--r--  1 

## Querying in PyTables

### Denormalized tables

In [27]:
h5denorm = "structuring/blosc-zstd-5-shuffle.h5"
h5file = tables.open_file(h5denorm)
h5lens = h5file.root.lens

In [28]:
h5lens

/lens (Table(1000209,), shuffle, blosc:zstd(5)) ''
  description := {
  "user_id": Int32Col(shape=(), dflt=0, pos=0),
  "rating": Int8Col(shape=(), dflt=0, pos=1),
  "unix_timestamp": Int64Col(shape=(), dflt=0, pos=2),
  "title": StringCol(itemsize=100, shape=(), dflt=b'', pos=3),
  "genres": StringCol(itemsize=50, shape=(), dflt=b'', pos=4)}
  byteorder := 'little'
  chunkshape := (402,)

In [29]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

CPU times: user 897 ms, sys: 141 ms, total: 1.04 s
Wall time: 968 ms


In [30]:
ratings

[0, 4, 15, 28, 18, 3]

In [31]:
h5file.close()

### Normalized tables

In [64]:
h5norm = "compression/blosc-zstd-5-shuffle.h5"
h5file = tables.open_file(h5norm)
h5ratings = h5file.root.ratings
h5movies = h5file.root.movies

In [65]:
h5ratings

/ratings (Table(1000209,), shuffle, blosc:zstd(5)) ''
  description := {
  "user_id": Int32Col(shape=(), dflt=0, pos=0),
  "movie_id": Int32Col(shape=(), dflt=0, pos=1),
  "rating": Int8Col(shape=(), dflt=0, pos=2),
  "unix_timestamp": Int64Col(shape=(), dflt=0, pos=3)}
  byteorder := 'little'
  chunkshape := (7710,)

In [66]:
h5movies

/movies (Table(3883,), shuffle, blosc:zstd(5)) ''
  description := {
  "movie_id": Int32Col(shape=(), dflt=0, pos=0),
  "title": StringCol(itemsize=100, shape=(), dflt=b'', pos=1),
  "genres": StringCol(itemsize=50, shape=(), dflt=b'', pos=2)}
  byteorder := 'little'
  chunkshape := (425,)

In [67]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

CPU times: user 293 ms, sys: 40.4 ms, total: 334 ms
Wall time: 319 ms


In [68]:
ratings

[0, 4, 15, 28, 18, 3]

In [69]:
h5file.close()

So, the query in the normalized version is more than 2~3x faster than using the denormalized file.  However, this is just a simple example, and in general experimentation should be done so as to determine the best layout for your data.

## Indexing

### Denormalized case

In [49]:
## Copy the original PyTables table into another file
import shutil
h5idx = "movielens-denorm-indexed.h5"
if os.path.exists(h5idx):
    os.unlink(h5idx)
shutil.copyfile(h5denorm, h5idx)

'movielens-denorm-indexed.h5'

In [50]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx, mode="a")

In [51]:
# Create an index for the 'title' column
h5lens = h5i.root.lens
blosc_filter = tables.Filters(complevel=9, complib="blosc")
%time h5lens.cols.title.create_csindex(filters=blosc_filter)

CPU times: user 1.44 s, sys: 287 ms, total: 1.73 s
Wall time: 1.74 s


1000209

In [52]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

CPU times: user 8.32 ms, sys: 2.2 ms, total: 10.5 ms
Wall time: 14 ms


In [53]:
ratings

[0, 4, 15, 28, 18, 3]

In [54]:
# Create an index for the rating column
%time h5lens.cols.rating.create_csindex(filters=blosc_filter)

CPU times: user 423 ms, sys: 37.7 ms, total: 460 ms
Wall time: 480 ms


1000209

In [55]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

CPU times: user 4.19 ms, sys: 952 µs, total: 5.15 ms
Wall time: 4.08 ms


In [56]:
ratings

[0, 4, 15, 28, 18, 3]

In [57]:
h5i.close()

### Normalized case

In [73]:
## Copy the original PyTables table into another file
import shutil
h5idx = "movielens-norm-indexed.h5"
if os.path.exists(h5idx):
    os.unlink(h5idx)
shutil.copyfile(h5norm, h5idx)

'movielens-norm-indexed.h5'

In [74]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx, mode="a")
h5ratings = h5i.root.ratings
h5movies = h5i.root.movies

In [75]:
# Create an index for the rating column
blosc_filter = tables.Filters(complevel=9, complib="blosc")
%time h5ratings.cols.rating.create_csindex(filters=blosc_filter)

CPU times: user 330 ms, sys: 33 ms, total: 362 ms
Wall time: 372 ms


1000209

In [76]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

CPU times: user 330 ms, sys: 42.1 ms, total: 372 ms
Wall time: 335 ms


In [77]:
ratings

[0, 4, 15, 28, 18, 3]

In [78]:
# Create an index for the movie_id column
%time h5ratings.cols.movie_id.create_csindex(filters=blosc_filter)

CPU times: user 339 ms, sys: 31.7 ms, total: 370 ms
Wall time: 386 ms


1000209

In [79]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

CPU times: user 24.3 ms, sys: 2.53 ms, total: 26.9 ms
Wall time: 24.9 ms


In [80]:
ratings

[0, 4, 15, 28, 18, 3]

In [81]:
h5i.close()

In [83]:
!du -sh movielens* | sort -nr

9.9M	movielens-denorm-indexed.h5
9.6M	movielens-norm-indexed.h5
5.8M	movielens-1m


## Exercise

We have not created an index for the title for the normalized case.  Create such an index and determine if there is a noticeable speed-up or not.  Explain why you think that is the case.  Note: the times for a cold query can be **significatively** different from a hot query.

In [84]:
## Copy the original PyTables table into another file
import shutil
h5idx2 = "movielens-norm-indexed2.h5"
if os.path.exists(h5idx2):
    os.unlink(h5idx2)
shutil.copyfile(h5idx, h5idx2)

'movielens-norm-indexed2.h5'

In [85]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx2, mode="a")
h5ratings = h5i.root.ratings
h5movies = h5i.root.movies

In [86]:
# Create an index for the movie_id column
%time h5movies.cols.title.create_csindex(filters=blosc_filter)

CPU times: user 13.3 ms, sys: 2.46 ms, total: 15.7 ms
Wall time: 14.8 ms


3883

In [87]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

CPU times: user 204 ms, sys: 27.1 ms, total: 231 ms
Wall time: 227 ms


In [88]:
ratings

[0, 4, 15, 28, 18, 3]

In [89]:
h5i.close()

In [90]:
!du -sh movielens* | sort -nr

9.9M	movielens-denorm-indexed.h5
9.7M	movielens-norm-indexed2.h5
9.6M	movielens-norm-indexed.h5
5.8M	movielens-1m
